In [124]:
import pandas as pd
import numpy as np
import sklearn 
import os
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler



In [2]:
neo=pd.read_csv(r"C:\Users\sujal\PycharmProjects\logistic_regression\dataset\nearest-earth-objects(1910-2024).csv")

In [3]:
neo.head()

,neo_id,name,absolute_magnitude,estimated_diameter_min,estimated_diameter_max,orbiting_body,relative_velocity,miss_distance,is_hazardous
0,2162117,162117 (1998 SD15),19.14,0.394962,0.883161,Earth,71745.401048,5.814362e+07,False
1,2349507,349507 (2008 QY),18.50,0.530341,1.185878,Earth,109949.757148,5.580105e+07,True
2,2455415,455415 (2003 GA),21.45,0.136319,0.304818,Earth,24865.506798,6.720689e+07,False
3,3132126,(2002 PB),20.63,0.198863,0.444672,Earth,78890.076805,3.039644e+07,False
4,3557844,(2011 DW),22.70,0.076658,0.171412,Earth,56036.519484,6.311863e+07,False


In [5]:
from sklearn.pipeline import Pipeline,make_pipeline

In [6]:
neo=neo.drop(columns=['name','neo_id'])

In [7]:
neo.drop(columns=['orbiting_body'], inplace=True)

In [8]:
neo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338199 entries, 0 to 338198
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   absolute_magnitude      338171 non-null  float64
 1   estimated_diameter_min  338171 non-null  float64
 2   estimated_diameter_max  338171 non-null  float64
 3   relative_velocity       338199 non-null  float64
 4   miss_distance           338199 non-null  float64
 5   is_hazardous            338199 non-null  bool   
dtypes: bool(1), float64(5)
memory usage: 13.2 MB


In [11]:
neo['average_diameter'] = (neo['estimated_diameter_min'] + neo['estimated_diameter_max']) / 2


neo.drop(['estimated_diameter_min', 'estimated_diameter_max'], axis=1, inplace=True)

In [12]:
neo.info()
from sklearn.compose import ColumnTransformer

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338199 entries, 0 to 338198
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   absolute_magnitude  338171 non-null  float64
 1   relative_velocity   338199 non-null  float64
 2   miss_distance       338199 non-null  float64
 3   is_hazardous        338199 non-null  bool   
 4   average_diameter    338171 non-null  float64
dtypes: bool(1), float64(4)
memory usage: 10.6 MB


In [156]:




from sklearn import set_config
from sklearn.linear_model import LogisticRegression
set_config(display="diagram")
trf1 = ColumnTransformer([
    ("impute_abs_magnitude", SimpleImputer(strategy="median"), [0]),
    ("impute_average_diameter", SimpleImputer(strategy="median"), [3])
], remainder="passthrough")

trf2 = ColumnTransformer([
    ('scaling', RobustScaler(), [0,1,2,3])
], remainder='passthrough')

# Create the pipeline
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('classifier', LogisticRegression())
])


In [157]:
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_abs_magnitude',
                                                  SimpleImputer(strategy='median'),
                                                  [0]),
                                                 ('impute_average_diameter',
                                                  SimpleImputer(strategy='median'),
                                                  [3])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaling', RobustScaler(),
                                                  [0, 1, 2, 3])])),
                ('classifier', LogisticRegression())])

In [158]:
encoder=LabelEncoder()

In [159]:
neo['is_hazardous']=encoder.fit_transform(neo['is_hazardous'])

In [160]:
neo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338199 entries, 0 to 338198
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   absolute_magnitude  338171 non-null  float64
 1   relative_velocity   338199 non-null  float64
 2   miss_distance       338199 non-null  float64
 3   is_hazardous        338199 non-null  int64  
 4   average_diameter    338171 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 12.9 MB


In [161]:
from sklearn.model_selection import train_test_split

X=neo[['absolute_magnitude','relative_velocity','miss_distance','average_diameter']]
y=neo['is_hazardous']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.info()
y_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 236739 entries, 312448 to 121958
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   absolute_magnitude  236718 non-null  float64
 1   relative_velocity   236739 non-null  float64
 2   miss_distance       236739 non-null  float64
 3   average_diameter    236718 non-null  float64
dtypes: float64(4)
memory usage: 9.0 MB
<class 'pandas.core.series.Series'>
Index: 236739 entries, 312448 to 121958
Series name: is_hazardous
Non-Null Count   Dtype
--------------   -----
236739 non-null  int64
dtypes: int64(1)
memory usage: 3.6 MB


In [162]:
pipe.fit(X_train,y_train)


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_abs_magnitude',
                                                  SimpleImputer(strategy='median'),
                                                  [0]),
                                                 ('impute_average_diameter',
                                                  SimpleImputer(strategy='median'),
                                                  [3])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaling', RobustScaler(),
                                                  [0, 1, 2, 3])])),
                ('classifier', LogisticRegression())])

In [171]:
pipe.fit(X_train, y_train)
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Predict on the test data using the pipeline
y_test_pred = pipe.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_test_pred)
conf_matrix = confusion_matrix(y_test, y_test_pred)
class_report = classification_report(y_test, y_test_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 0.8715454366252711
Confusion Matrix:
[[87486   982]
 [12051   941]]
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.99      0.93     88468
           1       0.49      0.07      0.13     12992

    accuracy                           0.87    101460
   macro avg       0.68      0.53      0.53    101460
weighted avg       0.83      0.87      0.83    101460


In [172]:
print(f"Accuracy is {accuracy}")

Accuracy is 0.8715454366252711
